In [22]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"4.xlsx", header=[0], index_col = [0])

# print(records)

path = "Data_File/ismt_yices2"
folder_list = os.listdir(path)
for filefolder in folder_list:
    file_path = path + "/" + filefolder
    file_list = os.listdir(file_path)
    solved = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0, 'ismt': 0, 'yices2': 0}
    times = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0, 'ismt': 0, 'yices2': 0}
    for filename in tqdm(file_list, desc = filefolder):
        if filename[-3:] != 'log':
            continue
        with open(os.path.join(file_path, filename)) as f:
            while f.readline():
                f.readline()
                flag = 'none'
                result = f.readline()
                t = 0
                while(result[:2] != 'is' and result[:2] != 'Fi' and result[:2] != '..'):
                    result = f.readline()
                    t += 1
                    if(t > 100):
                        break
                if(t > 100):
                    break
                if result[:4] == 'ismt':
                    flag = 'ismt'
                    info = f.readline()
                    if result[6] == 's':
                        result = 'sat'
                    elif result[6] == 'u':
                        result = 'unsat'
                elif result[:2] == '..':
                    info = result
                    result = 'error'
                elif result[:2] == 'Fi':
                    result = f.readline()
                    if result[:2] == '..':
                        info = result
                        result = 'timeout'
                    elif result[:2] == 'fi':
                        info = f.readline()
                        flag = 'yices2'
                        if result[14] == 's':
                            result = 'sat'
                        elif result[14] == 'u':
                            result = 'unsat'
                    elif result[:4] == 'unkn':
                        info = f.readline()
                        result = 'unknow'
                    else:
                        break
                info = info.split('../../')[-1]
                info = '../../' + info
                info = info.split('/')
                time = int(info[-1].split(' : ')[1][:-5])
                info[-1] = info[-1].split(' : ')[0]
                if result[:3] == 'sat':
                    solved['total'] += 1
                    times['total'] += time
                    solved[flag] += 1
                    times[flag] += time
                    if info[5] == '20170427-VeryMax':
                        if(len(info) == 5):
                            solved['SAT14'] += 1
                            times['SAT14'] += time
                        else:
                            solved[info[6]] += 1
                            times[info[6]] += time
                    elif info[5] == 'UltimateAutomizer' or info[5] == 'UltimateLassoRanker':
                        solved['LassoRanker'] += 1
                        times['LassoRanker'] += time
                    elif info[5] == '20210219-Dartagnan':
                        solved['Dartagnan'] += 1
                        times['Dartagnan'] += time
                    elif info[5] != '2019-ezsmt':
                        solved[info[5]] += 1
                        times[info[5]] += time
    records.loc[records['name'] == filefolder + '_solved', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total', 'ismt_solved', 'yices2_solved']] = [solved['AProVE'], solved['calypto'], solved['Dartagnan'], solved['LassoRanker'], solved['leipzig'], solved['mcm'], solved['CInteger'], solved['ITS'], solved['SAT14'], solved['total'], solved['ismt'], solved['yices2']]
    records.loc[records['name'] == filefolder + '_time', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total', 'ismt_solved', 'yices2_solved']] = [times['AProVE'], times['calypto'], times['Dartagnan'], times['LassoRanker'], times['leipzig'], times['mcm'], times['CInteger'], times['ITS'], times['SAT14'], times['total'], times['ismt'], times['yices2']]
print("Waiting...")
records.to_excel('Result/4_1.xlsx')
print("Done")

10: 100%|██████████| 49/49 [00:00<00:00, 836.48it/s]


Waiting...
Done
